In [63]:
import json
import http.client
import time
# import boto3
from datetime import datetime
import pandas as pd

In [56]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [57]:
# GLOBAL VARIBALE CONTAINING THE LAST ID NUMBER FOR EACH TABLE
country_last_id = 0
state_last_id = 0
location_last_id = 0
time_last_id = 0
forecast_last_id = 0
observation_last_id = 0
fact_last_id = 0


In [58]:
# we may also need to make a list of timezones that correlate with each city if we want to expand the system beyond cities in NSW

cities = [
    # 'Albury',
    # 'Coffs%20Harbour',
    'Sydney',
    'Broken%20Hill',
    'Goulburn',
    'Dubbo',
    # 'Newcastle',
    'Orange',
    'Wagga%20Wagga',
    'Maitland',
    'Wollongong',
    'Cessnock',
    # 'Griffith',
    # 'Armidale',
    # 'Grafton',
    # 'Queanbeyan',
    # 'Lismore',
    # 'Lithgow',
    # 'Nowra',
    # 'Tamworth',
    # 'Bathurst',
    # 'Ulladulla',
    # 'Taree',
    # 'Batemans Bay',
    # 'Port Macquarie',
    # 'Parkes',
    # 'Ballina',
    # 'Muswellbrook',
    # 'Kempsey',
    # 'Bowral',
    # 'Central Coast',
    # 'Kurri Kurri',
    # 'Singleton',
    # 'Tweed Heads',
    # 'Raymond Terrace',
    # 'Wauchope',
    # 'Mudgee',
    # 'Gunnedah',
    # 'Moree',
    # 'Byron Bay',
    # 'Inverell',
    # 'Casino',
    # 'Cooma',
    # 'Katoomba', # couldn't find on the api explorer
    # 'Leeton',
    # 'Deniliquin',
    # 'Narrabri',
    # 'Yamba',
    # 'Cowra',
    # 'Yass',
    ]

# the index for each dataframe will serve as the id number
location_df_base = pd.DataFrame(columns = ["state_id", "location_name", "location_lat", "location_long"])
state_df_base = pd.DataFrame(columns = ["count_id", "state_name"])
country_df_base = pd.DataFrame(columns = ["country_name"])

# time_df = pd.DataFrame(columns = ["time_time", "time_date"])
time_df_base = pd.DataFrame(columns = ["time_minutes", "time_hours", "time_day", "time_month", "time_year"])

forecast_df_base = pd.DataFrame(columns = ["time_minutes", "time_hours", "time_day", "time_month", "time_year"])

observation_df_base = pd.DataFrame(columns = ["observation_condition", "observation_temp_c", "observation_daytime", "observation_wind_kph", "observation_gust_kph", "observation_wind_direction", "observation_humidity", "observation_pressure_mb", "observation_cloudy", "observation_UV"])

fact_df_base = pd.DataFrame(columns = ["location_id", "observation_id", "time_id", "forecast_id"])
# if forecast_id is not empty, then the fact is of a forecasted weather event

# ^^^^^^^^^^
# These are the dataframe which each row will be appended to, must use pd.concat([df1, df2])



# Order of operations
1. Generate locations tables from list of locations (cites/countries) can query to extract province (state), timezone, and coordinate data
2. Create time entry using 15 minute intervals until the previous time of running
3. Create obervation table for each time interval (combine into fact table)
4. create forecasted oberservation table (combine into fact table)

In [66]:
# Time is created at the start of the script
current_time = datetime.now()
time_df = pd.DataFrame({"time_minutes":current_time.minute, "time_hours":current_time.hours, "time_day":current_time.day, "time_month":current_time.month, "time_year":current_time.year}, index=[time_last_id+1])
time_last_id += 1

time_df_base = pd.concat([time_df_base, time_df])

for city in cities:
    print(city)
    conn = http.client.HTTPSConnection("api.weatherapi.com")
    conn.request("GET", '/v1/current.json?key=30dc2bf00de44d26a5862046230803&q=' + city + '&aqi=no')
    res = conn.getresponse()
    data = res.read()
    decodedData = data.decode("utf-8")
    decodedData_json = json.loads(decodedData)

    observation_df = pd.DataFrame({"observation_condition":decodedData_json["current"]["condition"]["text"], "observation_temp_c":decodedData_json["current"]["temp_c"], "observation_daytime":decodedData_json["current"]["is_day"], "observation_wind_kph":decodedData_json["current"]["wind_kph"], "observation_gust_kph":decodedData_json["current"]["wind_kph"], "observation_wind_direction":decodedData_json["current"]["wind_dir"], "observation_humidity":decodedData_json["current"]["humidity"], "observation_pressure_mb":decodedData_json["current"]["pressure_mb"], "observation_cloudy":decodedData_json["current"]["cloud"], "observation_UV":decodedData_json["current"]["uv"]}, index = [observation_last_id+1])
    observation_last_id += 1
    observation_df_base = pd.concat([observation_df_base, observation_df])

    fact_df = pd.DataFrame({"location_id":city, "observation_id":observation_last_id, "time_id":time_last_id, "forecast_id":""}, index=[fact_last_id+1])
    fact_last_id += 1
    fact_df_base = pd.concat([fact_df_base, fact_df])


# we need to adjust how the json is read in
# location_df = pd.DataFrame(data = decodedData_json["location"],  index = [location_last_id + 1])


state_df = pd.DataFrame(columns = ["count_id", "state_name"])
country_df = pd.DataFrame(columns = ["country_name"])
location_last_id += 1


AttributeError: 'datetime.datetime' object has no attribute 'hours'

4

In [ ]:
fact_df_base

In [ ]:
observation_df_base

In [ ]:
# The locations can be added to the database manually using the script below
# find the state id needs to be done, this could be done using AWS tools?

state_last_id = 0
location_last_id = 0
country_last_id = 0

location_df = pd.DataFrame(columns = ["state_id", "location_name", "location_lat", "location_long"])
state_df = pd.DataFrame(columns = ["count_id", "state_name"])
country_df = pd.DataFrame(columns = ["country_name"])

location_df = pd.DataFrame([ ,decodedData_json["current"], decodedData_json["current"], decodedData_json["current"]] , columns = ["state_id", "location_name", "location_lat", "location_long"])

In [30]:
decodedData_json["current"]
# decodedData_json["location"]

{'last_updated_epoch': 1682334900,
 'last_updated': '2023-04-24 21:15',
 'temp_c': 18.0,
 'temp_f': 64.4,
 'is_day': 0,
 'condition': {'text': 'Partly cloudy',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
  'code': 1003},
 'wind_mph': 5.6,
 'wind_kph': 9.0,
 'wind_degree': 280,
 'wind_dir': 'W',
 'pressure_mb': 1032.0,
 'pressure_in': 30.47,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'humidity': 88,
 'cloud': 75,
 'feelslike_c': 18.0,
 'feelslike_f': 64.4,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'uv': 1.0,
 'gust_mph': 11.0,
 'gust_kph': 17.6}

In [36]:
# For observation data
# Get id number for location using AWS tools
print(type(decodedData_json["current"]))




forecast_df = pd.DataFrame(columns = ["time_minutes", "time_hours", "time_day", "time_month", "time_year"])
# Same as above, but for forecast data



<class 'dict'>
   time_minutes  time_hours  time_day  time_month  time_year
7            18          21        24           4       2023
  observation_condition  observation_temp_c  observation_daytime  \
4         Partly cloudy                18.0                    0   

   observation_wind_kph  observation_gust_kph observation_wind_direction  \
4                   9.0                   9.0                          W   

   observation_humidity  observation_pressure_mb  observation_cloudy  \
4                    88                   1032.0                  75   

   observation_UV  
4             1.0  


,0
0,1
1,2
2,3
3,4


# Note that: all times in utc!

In [12]:

# location_df["utc"] = location_df["localtime"].apply(pd.to_datetime)
location_df

,name,region,country,lat,lon,tz_id,localtime_epoch,localtime
Sydney,Sydney,New South Wales,Australia,-33.88,151.22,Australia/Sydney,1682128538,2023-04-22 11:55


In [6]:
# location_df['day'] = location_df["utc"].dt.day
# location_df['month'] = location_df["utc"].dt.month
# location_df['year'] = location_df["utc"].dt.year
# location_df['hour'] = location_df["utc"].dt.hour
# location_df['minute'] = location_df["utc"].dt.minute

In [44]:
location_df = location_df[["name", "region", "country", "lat", "lon"]]

,name,region,country,lat,lon,tz_id,localtime_epoch,localtime,utc,year,month,day,hour,minute
Sydney,Sydney,New South Wales,Australia,-33.88,151.22,Australia/Sydney,1681960283,2023-04-20 13:11,2023-04-20 13:11:00,2023,4,20,13,11


In [17]:
country_df = location_df[["country"]]
country_df = country_df.reset_index().drop(columns="index")
country_df

,country
0,Australia
